In [33]:
#CIFAR-10 classifier	3-channel images, overfitting	CIFAR-10

In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset,DataLoader
import torchvision
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F

In [35]:
means = [0.4914, 0.4822, 0.4465]
stds = [0.2023, 0.1994, 0.2010]

train_transform=transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean =means,
                          std =stds)
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=means,
                         std=stds)
])

train_dataset=torchvision.datasets.CIFAR10(root='./data',train=True,transform=train_transform,download=True)
test_dataset=torchvision.datasets.CIFAR10(root='./data',train=False,transform=test_transform,download=True)
train_loader=DataLoader(train_dataset,batch_size=64,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=64,shuffle=True)

In [36]:
class CIFAR10classifier(nn.Module):
    def __init__(self,hidden_size,output_size):
        super().__init__()
        self.conv1=nn.Conv2d(in_channels=3,out_channels=16,kernel_size=3,padding=1)
        self.pool=nn.MaxPool2d(kernel_size=2,stride=2)
        self.conv2=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,padding=1)
        self.pool2=nn.MaxPool2d(kernel_size=2,stride=2)
        self.flattened_size=32*8*8
        self.fc1=nn.Linear(self.flattened_size,hidden_size)
        self.fc2=nn.Linear(hidden_size,output_size)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self,X):
        X=self.pool(F.relu(self.conv1(X)))
        X=self.pool2(F.relu(self.conv2(X)))
        X=torch.flatten(X,1)
        X=F.relu(self.fc1(X))
        X = self.dropout(X)
        X=self.fc2(X)
        return X

In [37]:
hidden_size=64
output_size=10
model=CIFAR10classifier(hidden_size,output_size)
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)

In [38]:
inputs, labels = next(iter(train_loader))
outputs = model(inputs)
print("Output shape:", outputs.shape)
print("Loss:", criterion(outputs, labels))


Output shape: torch.Size([64, 10])
Loss: tensor(2.3027, grad_fn=<NllLossBackward0>)


In [40]:
no_epochs=30
model.train()
for epochs in range(no_epochs):
    running_loss=0
    for inputs,labels in train_loader:
        outputs=model(inputs)
        optimizer.zero_grad()
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
    average_loss=running_loss/len(train_loader)
    print(f"epoch {epochs+1}, loss:{average_loss}")


epoch 1, loss:1.1497570060555586
epoch 2, loss:1.1387497766700851
epoch 3, loss:1.1349467783023024
epoch 4, loss:1.1270648511626837
epoch 5, loss:1.1196828714721954
epoch 6, loss:1.1189947209089919
epoch 7, loss:1.1035489160996264
epoch 8, loss:1.1047630720888562
epoch 9, loss:1.1015898047963066
epoch 10, loss:1.0919435572288836
epoch 11, loss:1.0888997932224322
epoch 12, loss:1.0850715773642217
epoch 13, loss:1.0838130528054883
epoch 14, loss:1.0825447516368174
epoch 15, loss:1.0754409741867534
epoch 16, loss:1.072737330289753
epoch 17, loss:1.069023007093488
epoch 18, loss:1.069728509773074
epoch 19, loss:1.0646939910281346
epoch 20, loss:1.0635364032767314
epoch 21, loss:1.0610731406437466
epoch 22, loss:1.0578655663048824
epoch 23, loss:1.0500835972978635
epoch 24, loss:1.0536528472095499
epoch 25, loss:1.0478688867195793
epoch 26, loss:1.0422712423459952
epoch 27, loss:1.0459471079699523
epoch 28, loss:1.0423386609157943
epoch 29, loss:1.0383595132919223
epoch 30, loss:1.040187271

In [41]:
torch.save(model.state_dict(), 'cifar10_classifier.pth')
print("✅ Model saved as 'cifar10_classifier.pth'")

✅ Model saved as 'cifar10_classifier.pth'


In [42]:
model.eval()
with torch.no_grad():
    total=0
    correct=0
    for inputs,labels in test_loader:
        outputs=model(inputs)
        _,predicted=torch.max(outputs.data,1)
        total+=labels.size(0)
        correct+=(predicted==labels).sum().item()
    print(outputs.shape)
    accuracy = 100 * correct / total
    print(f'Accuracy of the network on the 10000 test images: {accuracy:.2f}%')






torch.Size([16, 10])
Accuracy of the network on the 10000 test images: 71.49%


In [43]:
image=Image.open("C:/Users/LENOVO/Downloads/horseCIFAR.jpg").convert('RGB')
transform_img=transforms.Compose([transforms.Resize((32,32)),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.4914,0.4822,0.4465],
                                                       std=[0.2023,0.1994,0.2010])
                                                       ])
input_tesnor=transform_img(image)
input_tensor=input_tesnor.unsqueeze(0)

with torch.no_grad():
    outputs = model(input_tensor)
    _, predicted = torch.max(outputs, 1)
    predicted_class = predicted.item()
classes=['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
print('predicted class:',classes[predicted_class])

predicted class: horse
